In [62]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import ccxt
import itertools
import ipywidgets
from datetime import datetime, timedelta
from numba import njit


In [67]:
seed = 42
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'ADAUSDT', 'LTCUSDT']
start_date = datetime(2020, 3, 24)
end_date = datetime(2023, 1, 19)
time_delta = end_date - start_date
window_len = timedelta(days=180)
window_count = 400
exit_types = ['SL', 'TS', 'TP', 'Random', 'Holding']
step = 0.01  # in %
stops = np.arange(step, 1 + step, step)

# vbt.settings.layout['template'] = vbt.settings.dark_template

vbt.settings.plotting["layout"]["template"] = "vbt_dark"
vbt.settings.portfolio['freq'] = '1d'
vbt.settings.portfolio['init_cash'] = 100.  # in $
vbt.settings.portfolio['fees'] = 0.0025  # in %
vbt.settings.portfolio['slippage'] = 0.0025  # in %


In [68]:
print(pd.Series({
    'Start date': start_date,
    'End date': end_date,
    'Time period (days)': time_delta.days,
    'Assets': len(symbols),
    'Window length': window_len,
    'Windows': window_count,
    'Exit types': len(exit_types),
    'Stop values': len(stops),
    'Tests per asset': window_count * len(stops) * len(exit_types),
    'Tests per window': len(symbols) * len(stops) * len(exit_types),
    'Tests per exit type': len(symbols) * window_count * len(stops),
    'Tests per stop type and value': len(symbols) * window_count,
    'Tests total': len(symbols) * window_count * len(stops) * len(exit_types)
}))


Start date                       2020-03-24 00:00:00
End date                         2023-01-19 00:00:00
Time period (days)                              1031
Assets                                             6
Window length                      180 days, 0:00:00
Windows                                          400
Exit types                                         5
Stop values                                      100
Tests per asset                               200000
Tests per window                                3000
Tests per exit type                           240000
Tests per stop type and value                   2400
Tests total                                  1200000
dtype: object


In [69]:
cols = ['Open', 'Low', 'High', 'Close', 'Volume']
data = vbt.CCXTData.download(
    symbols,
    start=start_date,
    end=end_date,
    timeframe='1d',
    exchange='bybit'
    )
data.save('stop_sigs')
data = vbt.CCXTData.load('stop_sigs')
ohlcv_by_symbol = data.data


0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [70]:
ohlcv_by_symbol['BTCUSDT']

,Open,High,Low,Close,Volume
Open time,,,,,
2020-03-25 00:00:00+00:00,6500.0,6745.5,6500.0,6698.5,1809.520
2020-03-26 00:00:00+00:00,6698.5,6767.0,6512.0,6733.5,3904.964
2020-03-27 00:00:00+00:00,6733.5,6838.0,6235.0,6354.0,4605.347
2020-03-28 00:00:00+00:00,6354.0,6354.0,6010.0,6230.5,5750.959
2020-03-29 00:00:00+00:00,6230.5,6247.0,5858.0,5873.0,5347.238
...,...,...,...,...,...
2023-01-15 00:00:00+00:00,20952.5,21066.0,20555.0,20878.0,118541.184
2023-01-16 00:00:00+00:00,20878.0,21488.0,20600.0,21179.0,207040.534
2023-01-17 00:00:00+00:00,21179.0,21626.5,20824.5,21118.0,161002.365


In [71]:
print(ohlcv_by_symbol.keys())
print(ohlcv_by_symbol['BTCUSDT'].shape)
ohlcv_by_symbol['BTCUSDT'].vbt.ohlcv.plot()


dict_keys(['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'ADAUSDT', 'LTCUSDT'])
(1031, 5)


FigureWidget({
    'data': [{'close': array([ 6698.5,  6733.5,  6354. , ..., 21118. , 20674. , 21058.5]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([ 6745.5,  6767. ,  6838. , ..., 21626.5, 21673. , 21173.5]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([ 6500. ,  6512. ,  6235. , ..., 20824.5, 20385. , 20645. ]),
              'name': 'OHLC',
              'open': array([ 6500. ,  6698.5,  6733.5, ..., 21179. , 21118. , 20674. ]),
              'type': 'ohlc',
              'uid': '2a887adc-e841-4a97-a97d-384d5531588f',
              'x': array([datetime.datetime(2020, 3, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 3, 26, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 3, 27, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 1, 17, 0, 0, tzinfo=datetime.timezone.utc),
   

In [74]:
ohlcv = data.concat()
ohlcv.keys()
ohlcv['Open'].shape

(1031, 6)

In [79]:
ohlcv_data={}
for k, v in ohlcv.items():
    ohlcv_data[k] = v.vbt.range_split(range_len=window_len.days, n=window_count)[0]
ohlcv_data['Open']

split_idx      0                                                1            \
symbol     BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT ETHUSDT   
0           6500.0     NaN     NaN     NaN     NaN     NaN   6733.5     NaN   
1           6698.5     NaN     NaN     NaN     NaN     NaN   6354.0     NaN   
2           6733.5     NaN     NaN     NaN     NaN     NaN   6230.5     NaN   
3           6354.0     NaN     NaN     NaN     NaN     NaN   5873.0     NaN   
4           6230.5     NaN     NaN     NaN     NaN     NaN   6389.5     NaN   
..             ...     ...     ...     ...     ...     ...      ...     ...   
175        10778.5     NaN     NaN     NaN     NaN     NaN  10928.0     NaN   
176        10947.0     NaN     NaN     NaN     NaN     NaN  10917.0     NaN   
177        10928.0     NaN     NaN     NaN     NaN     NaN  11071.0     NaN   
178        10917.0     NaN     NaN     NaN     NaN     NaN  10912.0     NaN   
179        11071.0     NaN     NaN     NaN     NaN     NaN  10408.0     NaN   

split_idx                  ...     398                              399  \
symbol    XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT   
0             NaN     NaN  ...  0.3664  265.30  0.4990   58.02  22434.5   
1             NaN     NaN  ...  0.3575  262.20  0.4827   56.06  22573.5   
2             NaN     NaN  ...  0.3590  258.90  0.5167   56.35  21301.0   
3             NaN     NaN  ...  0.3586  261.45  0.5118   58.35  21232.0   
4             NaN     NaN  ...  0.3357  244.35  0.4749   53.93  22938.0   
..            ...     ...  ...     ...     ...     ...     ...      ...   
175           NaN     NaN  ...  0.3747  287.45  0.3291   86.04  20952.5   
176           NaN     NaN  ...  0.3856  293.60  0.3454   86.23  20878.0   
177           NaN     NaN  ...  0.3954  305.20  0.3525   87.89  21179.0   
178           NaN     NaN  ...  0.3847  302.05  0.3508   87.43  21118.0   
179           NaN     NaN  ...  0.3856  298.95  0.3503   85.91  20674.0   

split_idx                                           
symbol     ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0          1547.80  0.3590  258.90  0.5167   56.35  
1          1597.20  0.3586  261.45  0.5118   58.35  
2          1440.00  0.3357  244.35  0.4749   53.93  
3          1447.95  0.3368  249.25  0.4675   53.89  
4          1635.10  0.3592  271.55  0.5114   59.02  
..             ...     ...     ...     ...     ...  
175        1549.85  0.3954  305.20  0.3525   87.89  
176        1552.35  0.3847  302.05  0.3508   87.43  
177        1576.50  0.3856  298.95  0.3503   85.91  
178        1564.59  0.3868  298.90  0.3455   87.00  
179        1510.65  0.3781  286.40  0.3264   82.68  

[180 rows x 2400 columns]

In [78]:
ohlcv_indexs = pd.Series(ohlcv['Open'].vbt.range_split(range_len=window_len.days, n=window_count)[1])
ohlcv_indexs

0      DatetimeIndex(['2020-03-25 00:00:00+00:00', '2...
1      DatetimeIndex(['2020-03-27 00:00:00+00:00', '2...
2      DatetimeIndex(['2020-03-29 00:00:00+00:00', '2...
3      DatetimeIndex(['2020-03-31 00:00:00+00:00', '2...
4      DatetimeIndex(['2020-04-03 00:00:00+00:00', '2...
                             ...                        
395    DatetimeIndex(['2022-07-15 00:00:00+00:00', '2...
396    DatetimeIndex(['2022-07-18 00:00:00+00:00', '2...
397    DatetimeIndex(['2022-07-20 00:00:00+00:00', '2...
398    DatetimeIndex(['2022-07-22 00:00:00+00:00', '2...
399    DatetimeIndex(['2022-07-24 00:00:00+00:00', '2...
Length: 400, dtype: object

In [81]:
ohlcv_data['Open'].columns

MultiIndex([(  0, 'BTCUSDT'),
            (  0, 'ETHUSDT'),
            (  0, 'XRPUSDT'),
            (  0, 'BNBUSDT'),
            (  0, 'ADAUSDT'),
            (  0, 'LTCUSDT'),
            (  1, 'BTCUSDT'),
            (  1, 'ETHUSDT'),
            (  1, 'XRPUSDT'),
            (  1, 'BNBUSDT'),
            ...
            (398, 'XRPUSDT'),
            (398, 'BNBUSDT'),
            (398, 'ADAUSDT'),
            (398, 'LTCUSDT'),
            (399, 'BTCUSDT'),
            (399, 'ETHUSDT'),
            (399, 'XRPUSDT'),
            (399, 'BNBUSDT'),
            (399, 'ADAUSDT'),
            (399, 'LTCUSDT')],
           names=['split_idx', 'symbol'], length=2400)

In [100]:
entries = pd.DataFrame.vbt.signals.empty_like(ohlcv_data['Open'])
entries.iloc[0, : ] = True
entries

split_idx     0                                               1            \
symbol    BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT BTCUSDT ETHUSDT   
0            True    True    True    True    True    True    True    True   
1           False   False   False   False   False   False   False   False   
2           False   False   False   False   False   False   False   False   
3           False   False   False   False   False   False   False   False   
4           False   False   False   False   False   False   False   False   
..            ...     ...     ...     ...     ...     ...     ...     ...   
175         False   False   False   False   False   False   False   False   
176         False   False   False   False   False   False   False   False   
177         False   False   False   False   False   False   False   False   
178         False   False   False   False   False   False   False   False   
179         False   False   False   False   False   False   False   False   

split_idx                  ...     398                             399  \
symbol    XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT BTCUSDT   
0            True    True  ...    True    True    True    True    True   
1           False   False  ...   False   False   False   False   False   
2           False   False  ...   False   False   False   False   False   
3           False   False  ...   False   False   False   False   False   
4           False   False  ...   False   False   False   False   False   
..            ...     ...  ...     ...     ...     ...     ...     ...   
175         False   False  ...   False   False   False   False   False   
176         False   False  ...   False   False   False   False   False   
177         False   False  ...   False   False   False   False   False   
178         False   False  ...   False   False   False   False   False   
179         False   False  ...   False   False   False   False   False   

split_idx                                          
symbol    ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0            True    True    True    True    True  
1           False   False   False   False   False  
2           False   False   False   False   False  
3           False   False   False   False   False  
4           False   False   False   False   False  
..            ...     ...     ...     ...     ...  
175         False   False   False   False   False  
176         False   False   False   False   False  
177         False   False   False   False   False  
178         False   False   False   False   False  
179         False   False   False   False   False  

[180 rows x 2400 columns]

https://vectorbt.dev/api/signals/generators/#vectorbt.signals.generators.OHLCSTX

In [126]:
sl_ohlcstx = vbt.OHLCSTX.run(
    entries, 
    open=ohlcv_data["Open"], 
    high=ohlcv_data["High"], 
    low=ohlcv_data["Low"], 
    close=ohlcv_data["Close"], 
    sl_stop=list(stops),
)